In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 10 kB/s 
     |████████████████████████████████| 367 kB 55.1 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 3.2 MB 23.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you 

In [ ]:
# NOTE: Ensure "Runtime -> Change runtime type" is set to TPU.


# fetch the code.
!git clone https://github.com/shawwn/gpt-2 -b tpu /content/gpt-2
import os
os.chdir('/content/gpt-2')

# install prerequisites.
!pip install regex
!pip install toposort
!python download_model.py 117M

# fetch a dataset. You can use your own. See http://gwern.net/GPT-2 for how to prepare one.
# This dataset consists of all Emacs Lisp code that ships with emacs as of August 2019.
# It was generated by concatenating together all .el files, separated by <|endoftext|>.
# Tab characters were converted to 8 spaces. This makes it possible for GPT-2 to learn the proper indentation of various Lisp forms.
!gsutil cp gs://sgappa-multi/gpt-2/elisp-untab-eot-v001.txt.npz .

# Note that there is currently no support for saving the trained model.
# Theoretically it might work, but you'll have to create your own storage bucket and pass in --storage_bucket gs://your-bucket/gpt-2/

# Also note that if you plan on increasing the context window on the larger models, you should pass --sample_length 1023. (It defaults to n_ctx - 1.)
# The reason is because the sampling process gets exponentially slower as sample_length increases, so a large model
# with a big sample_length might take hours to generate even one sample.

# That said, it's interesting to train a 117M with --n_ctx 2048 --sample_length 2047 and watch it generate samples 2x longer than standard GPT-2.
# (Sampling from 117M is *much* faster than the larger models, so you can get away with a larger sample_length during training.)


# TO RUN THIS: Uncomment one of the following examples, then evaluate this cell.


# Create a new 117M
#!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 1024 --n_embd 768 --n_head 12 --n_layer 12

# Create a new 345M
#!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 1024 --n_embd 1024 --n_head 16 --n_layer 24

# Create a new 1.5B
#!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 1024 --n_embd 1600 --n_head 25 --n_layer 48

# Create a new 117M with 2x the context
#!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 2048 --n_embd 768 --n_head 12 --n_layer 12

# Fine-tune OpenAI's 774M on a dataset (note: saving not yet supported)
!python3 download_model.py 774M
# !PYTHONPATH=src python ./train.py --dataset /content/drive/MyDrive/horror_sequences.npz --model_name 774M --batch_size 1 --save_every 100000 --sample_every 20 --init_tpu
!PYTHONPATH=src python ./train.py --dataset 6_genre_clean_training_data_v2.npz --model_name 774M --batch_size 1 --save_every 100000 --sample_every 20 --init_tpu

# /content/gpt-2/horror_sequences_flattened.npz

fatal: destination path '/content/gpt-2' already exists and is not an empty directory.
  0% 0/1 [00:00<?, ?it/s]
  0% 0/7 [00:00<?, ?it/s]

Fetching encoder.json:   0%|                                           | 0.00/1.04M [00:00<?, ?it/s]

Fetching encoder.json: 1.04Mit [00:00, 7.74Mit/s]                                                   

 14% 1/7 [00:00<00:01,  3.18it/s]

Fetching hparams.json: 1.00kit [00:00, 745kit/s]                                                    

 29% 2/7 [00:00<00:00,  5.18it/s]

Fetching vocab.bpe:   0%|                                               | 0.00/456k [00:00<?, ?it/s]

Fetching vocab.bpe: 457kit [00:00, 4.24Mit/s]                                                       

 43% 3/7 [00:00<00:00,  4.90it/s]

Fetching checkpoint: 1.00kit [00:00, 773kit/s]                                                      

 57% 4/7 [00:00<00:00,  5.79it/s]

Fetching model.ckpt.data-00000-of-00001:   0%|                          | 0.00/498M [00:00<?, ?it/s]

Fetchi

In [20]:
import numpy as np
from numpy import load


data = load('/content/drive/MyDrive/horror_sequences.npz')
lst = data.files
for item in lst:
    print(item)
    print(data[item])
data_np = np.array(data[item]).flatten()
print(data_np)

np.savez('horror_sequences_flattened.npz', data_np)
# ___final data looks same: [   7 1745  212 ...    9    5  341] 


# data = load('elisp-untab-eot-v001.txt.npz')
# lst = data.files
# for item in lst:
#     print(item)
#     print(data[item])

# lookslike: arr_0 [19510 45991   764 ...    91    29   198]"""


# with numpy.load('elisp-untab-eot-v001.txt.npz') as data:
#     a = data

# for item in data_np:
#     print(item)
#     print(data[item])


# with numpy.load('/content/drive/MyDrive/horror_sequences.npz') as data:
#     a = data['a']


arr_0
[[   7 1745  212 ... 1739  179  717]
 [1745  212 1744 ...  179  717    2]
 [ 212 1744    4 ...  717    2  718]
 ...
 [  23    4 1737 ...    1   95    9]
 [   4 1737 1738 ...   95    9    5]
 [1737 1738   26 ...    9    5  341]]
[   7 1745  212 ...    9    5  341]


In [25]:
!PYTHONPATH=src python train.py --dataset 6_genre_clean_training_data_v2.npz --run_name run_cleaned_training_data

unknown option --restore_from
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.


In [24]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()



convert_to_text = tokenizer.sequences_to_texts(a)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

ModuleNotFoundError: ignored

In [21]:
print(convert_to_text)

NameError: ignored

In [22]:
python encode.py 6_genre_clean_training_data_v2.txt 6_genre_clean_training_data_v2.npz

SyntaxError: ignored

In [23]:
!PYTHONPATH=src python encode.py 6_genre_clean_training_data_v2.txt 6_genre_clean_training_data_v2.npz

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 